In [662]:
import cv2
import numpy as np

In [663]:
def display(image, scale=0.5):
    shrunk = cv2.resize(image, (0,0), fx=scale, fy=scale)
    cv2.imshow("Screen Location", shrunk)

    if cv2.waitKey(1000) & 0xFF == ord('q'):
        cv2.destroyAllWindows()

In [664]:
billboard_colour = [150, 20, 20]
def strip_colours(image, preserve=billboard_colour):
    black_indices = np.where((image != preserve).all(axis=2))
    white_indices = np.where((image == preserve).all(axis=2))
    rtn = image.copy()
    rtn[black_indices] = [0,0,0]
    rtn[white_indices] = [255,255,255]
    return rtn

# noinspection PyDefaultArgument
def add_borders(image, border_color= [0,0,0], border_width=25):
    return cv2.copyMakeBorder(
        image,
        border_width,
        border_width,
        border_width,
        border_width,
        cv2.BORDER_CONSTANT,
        value=border_color
    )

def remove_borders(image, border_width=25):
  return image[border_width:-border_width, border_width:-border_width]

# noinspection PyShadowingNames
def get_largest_billboard_contour(image):
    edged_image = cv2.Canny(image, 30, 200)

    (contours, _) = cv2.findContours(
        edged_image.copy(),
        cv2.RETR_TREE,
        cv2.CHAIN_APPROX_SIMPLE
    )
    contours = sorted(
        contours,
        key=cv2.contourArea,
        reverse=True
    )[:10]


    best = (0,0)

    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        area = cv2.contourArea(contour)
        print(area, end="\t")
        print(len(approx))
        # if len(approx) == 4:
        if area > best[0]:
          best = (area, approx)

    return best[1] if best[0] > 0 else None

# noinspection PyShadowingNames
def get_contour_mask(image, contour):
    rtn = image.copy()
    cv2.drawContours(
        image=rtn,
        contours=[contour],
        contourIdx=-1,
        color=(255,255,255),
        thickness=cv2.FILLED
    )
    return rtn

In [665]:
source_image = cv2.imread("../resources/new/015_00010.png")
print(source_image.shape)

padded_source_image = add_borders(source_image)
display(padded_source_image)

binary_image = strip_colours(padded_source_image)
display(binary_image)

(1080, 1920, 3)


In [666]:
display_image = padded_source_image.copy()
contour = get_largest_billboard_contour(binary_image)

37642.5	4
37641.5	4
2413.5	4
2406.5	4
100.5	8
100.0	8
16.0	12
13.0	11
10.0	6
10.0	8


In [667]:
outlined = display_image.copy()
outlined = cv2.drawContours(
    outlined,
    [contour],
    -1,
    [0,255,0],
    3
)
display(outlined)

In [690]:
# noinspection PyShadowingNames
def get_perspective_warped_decal(background, decal, contour):
    src = np.float32([
        [0,0],
        [decal.shape[0], 0],
        decal.shape[:2],
        [0, decal.shape[1]],
    ])
    dst = np.float32(contour)
    m = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(
        decal_image,
        m,
        (background.shape[1], background.shape[0])
    )
    return warped

# noinspection PyShadowingNames
def get_affine_warped_decal(background, decal, contour):
    src = np.float32([
        [0,0],
        [decal.shape[0], 0],
        # decal.shape[:2],
        [0, decal.shape[1]],
    ])
    dst = np.float32([x[0] for i,x in enumerate(contour) if i!=2])
    m = cv2.getAffineTransform(src, dst)
    warped = cv2.warpAffine(
        decal_image,
        m,
        (background.shape[1], background.shape[0])
    )
    return warped

(1130, 1970, 3)
[[[1944  136]]

 [[1674  181]]

 [[1666  307]]

 [[1944  283]]]
[[[3888  419]]

 [[3618  464]]

 [[3610  590]]]


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wbmte9m7\opencv\modules\calib3d\src\fundam.cpp:380: error: (-215:Assertion failed) src.checkVector(2) == dst.checkVector(2) in function 'cv::findHomography'


In [794]:
def rotate(lst, x):
    return lst[-x:] + lst[:-x]
# noinspection PyShadowingNames
def get_homo_warped_decal(background, decal, contour):
    contour = [[x[0]] for x in contour]
    # contour = contour[
    contour = rotate(contour, 1)
    contour = np.float32(contour)

    h_decal, w_decal = decal.shape[:2]
    h_background, w_background = background.shape[:2]
    src = np.float32([
        [0, h_decal],
        [0,0],
        [w_decal, 0],
        [w_decal, h_decal],
    ])
    dst = np.float32(contour)
    h, mask = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)
    warped = cv2.warpPerspective(decal_image, h, (w_background, h_background))
    colors = [(0,255,0),(255,0,0),(255,255,0),(0,255,255)]
    for i, center in enumerate([x[0] for x in contour]):
        cv2.circle(warped, tuple(center), 20, colors[i], -1)
    return warped


decal_image = cv2.imread("../resources/new/decal_0.png")

warped = get_homo_warped_decal(outlined, decal_image, contour)
temp = cv2.addWeighted(outlined, 1, warped, 1, 1)
display(temp)

# warped = get_affine_warped_decal(outlined, decal_image, contour)
# temp = cv2.addWeighted(outlined, 0.5, warped, 0.5, 1)
# display(temp)

In [669]:
mask = get_contour_mask(display_image, contour)
display(mask)

In [670]:
# final = cv2.bitwise_xor(display_image, display_image, mask=mask)
# display(final)